In [1]:
# jupyter notebook
!jupyter nbextension enable --py --sys-prefix ipyleaflet

Enabling notebook extension jupyter-leaflet/extension...
      - Validating: ok


In [2]:
# jupyter lab
# !jupyter labextension install jupyter-leaflet
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager

In [2]:
import os
import sys
import time
from datetime import datetime
from datetime import timedelta
from itertools import chain
from itertools import cycle
import collections
from pprint import pprint
from io import StringIO
import pathlib

import numpy as np
import pandas as pd
from pandas.tools.plotting import parallel_coordinates
import matplotlib.pyplot as plt

#import urllib
#import webbrowser
#import codecs
from bs4 import BeautifulSoup
from xml.dom import minidom
from xml.etree import ElementTree as ET
#from lxml import etree as ET #Supports xpath syntax
import xmltodict 

import nltk
import re
import Levenshtein
import networkx as nx
import json

#from nominatim import Nominatim
#from mpl_toolkits.basemap import Basemap   #http://matplotlib.org/basemap/api/basemap_api.html
from matplotlib import cm
import geopy.geocoders as gg
from geopy.geocoders import Nominatim
import geopandas as gpd
import ipyleaflet
from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, WMSLayer, Marker, MarkerCluster

#import plotly as py
#import plotly.graph_objs as go
#import plotly.figure_factory as ff

from IPython.display import Image, HTML, IFrame, FileLink, FileLinks #needed to render in notebook
from IPython.core.display import display

#import pydotplus #Install http://www.graphviz.org/ & Instal https://pypi.python.org/pypi/pydotplus

#https://stackoverflow.com/questions/2148119/how-to-convert-an-xml-string-to-a-dictionary-in-python
#https://github.com/martinblech/xmltodict
#https://www.datacamp.com/community/tutorials/python-xml-elementtree
#https://stackabuse.com/reading-and-writing-xml-files-in-python/
#https://codereview.stackexchange.com/questions/117810/parsing-of-xml-in-python-using-minidom-and-inserting-data-into-a-database
#https://stackoverflow.com/questions/749796/pretty-printing-xml-in-python

In [4]:
pwd

'C:\\Users\\rsouza\\Documents\\Python Scripts\\exploreAT-Topothek.NLP'

Reading the XML Metadata files:

In [5]:
readme =  './TEI-XML-2018/README_English.txt'
print((open(readme, encoding="latin-1").read()))


Content of the XML files
XML are partially enriched with location data
* The location data are available as a tree in the respective entries
* Location data are available for those entries specified in helper_tables\placeName-bis Freq 10-nur Ö+STir - Tabellenblatt1.csv
* The basis are complete matches in the tag placeName

The complete place hierarchy can be found in helper_tables\listPlace.xml. This resulted in helper_tables\listPlaceBySigle.xml

There could be manual corrections to the data.


Processing Information on Places

In [6]:
placesfile =  './TEI-XML-2018/helper_tables/listPlace.xml'
#print((open(placesfile, encoding="utf-8").read()))
placestree = ET.parse(placesfile)
placesroot = placestree.getroot()
set([elem.tag for elem in placesroot.iter()])

{'{http://www.tei-c.org/ns/1.0}Bundesland',
 '{http://www.tei-c.org/ns/1.0}Gemeinde',
 '{http://www.tei-c.org/ns/1.0}Großregion',
 '{http://www.tei-c.org/ns/1.0}Kleinregion',
 '{http://www.tei-c.org/ns/1.0}Kommentar',
 '{http://www.tei-c.org/ns/1.0}Ort',
 '{http://www.tei-c.org/ns/1.0}Sigle_DB',
 '{http://www.tei-c.org/ns/1.0}idno',
 '{http://www.tei-c.org/ns/1.0}listPlace',
 '{http://www.tei-c.org/ns/1.0}note',
 '{http://www.tei-c.org/ns/1.0}place',
 '{http://www.tei-c.org/ns/1.0}placeName',
 '{http://www.tei-c.org/ns/1.0}record'}

In [7]:
placesdict = xmltodict.parse((open(placesfile, encoding="utf-8").read()))
print(placesdict.keys())
print(placesdict['listPlace'].keys())

odict_keys(['listPlace'])
odict_keys(['@xmlns', 'place'])


In [8]:
placeslist = placesdict['listPlace']['place']
state_bundesland = [p['placeName'] for p in placeslist]
print(state_bundesland)

['W.', 'STir.', 'OTir.', 'NTir.', 'Kä.', 'St.', 'Sa.', 'OÖ', 'NÖ', 'Bgl.']


In [9]:
placeslist[0]['listPlace']['place']['listPlace']['place']['listPlace']['place']['listPlace']['place'][0]['record']
#placeslist[1]['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place'][0]['record']
#placeslist[2]['listPlace']['place']['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place']['record']
#placeslist[3]['listPlace']['place'][0]['listPlace']['place'][0]['listPlace']['place']['listPlace']['place']['record']

OrderedDict([('Ort', 'Altmannsdorf'),
             ('Gemeinde', 'Wien'),
             ('Kleinregion', 'W.'),
             ('Großregion', 'W.'),
             ('Bundesland', 'W.'),
             ('Sigle_DB', None),
             ('Kommentar', None)])

In [11]:
identified_places.keys()

dict_keys(['W.', 'STir.', 'OTir.', 'NTir.', 'Kä.', 'St.', 'Sa.', 'OÖ', 'NÖ', 'Bgl.'])

In [12]:
df_places = pd.DataFrame(columns=['Bundesland', 'Ort'])
for key in identified_places.keys():
    df_temp = pd.DataFrame(identified_places[key], columns=['Ort'])
    df_temp['Bundesland'] = key
    df_places = pd.concat([df_places, df_temp], axis=0, sort=True)
df_places.head(30)

,Bundesland,Ort
0,W.,Altmannsdorf
1,W.,Aspern
2,W.,Atzgersdorf
3,W.,Breitensee
4,W.,Favoriten
5,W.,Gersthof
6,W.,Hernals
7,W.,Hietzing
8,W.,Inzersdorf
9,W.,Kalksburg


Using a geolocator to get the Coordinates

In [13]:
geolocator = gg.Nominatim(user_agent="test_application")

In [14]:
logradouro = identified_places['Bgl.'][0]
location = geolocator.geocode(logradouro)
print(location.address)
address = location.address
print((location.latitude, location.longitude))
latitude, longitude = location.latitude, location.longitude
print(location.raw)

Deutsch Jahrndorf, Bezirk Neusiedl, Burgenland, 2423, Österreich
(48.0116888, 17.1069611)
{'place_id': '197784614', 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'relation', 'osm_id': '303140', 'boundingbox': ['47.9708775', '48.031224', '17.053944', '17.160776'], 'lat': '48.0116888', 'lon': '17.1069611', 'display_name': 'Deutsch Jahrndorf, Bezirk Neusiedl, Burgenland, 2423, Österreich', 'class': 'boundary', 'type': 'administrative', 'importance': 0.523562468688337, 'icon': 'https://nominatim.openstreetmap.org/images/mapicons/poi_boundary_administrative.p.20.png'}


The next cells were used just for getting geographic information. The saved Pickle shall be used for the subsequent processing. 

In [49]:
df_places = pd.read_pickle("./df_places.pkl")
df_places.head(10)

,Bundesland,Ort,Coordinates,Adresses
0,W.,Altmannsdorf,"(10.4342455, 49.9358878)","Altmannsdorf, Gerolzhofen (VGem), Landkreis Sc..."
1,W.,Aspern,"(16.4847508, 48.2181854)","Aspern, Donaustadt, Wien, 1220, Österreich"
2,W.,Atzgersdorf,"(16.2954875, 48.1469838)","Atzgersdorf, Liesing, Wien, 1230, Österreich"
3,W.,Breitensee,"(10.5435061, 50.3531716)","Breitensee, Bad Königshofen i. Grabfeld (VGem)..."
4,W.,Favoriten,"(16.3828218364771, 48.1529937)","Favoriten, Wien, 1100, Österreich"
5,W.,Gersthof,None,None
6,W.,Hernals,"(16.2842142186391, 48.2354031)","Hernals, Wien, 1170, Österreich"
7,W.,Hietzing,"(16.2529858155656, 48.17854095)","Hietzing, Wien, 1130, Österreich"
8,W.,Inzersdorf,"(16.3527158778638, 48.14397215)","KG Inzersdorf, Liesing, Wien, 1230, Österreich"
9,W.,Kalksburg,"(16.2457624, 48.1376728)","Kalksburg, Liesing, Wien, 1230, Österreich"


In [100]:
df_places.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2981 entries, 0 to 2980
Data columns (total 4 columns):
Bundesland     2981 non-null object
Ort            2981 non-null object
Coordinates    2951 non-null object
Adresses       2952 non-null object
dtypes: object(4)
memory usage: 93.2+ KB


In [101]:
df_places.reset_index(inplace=True, drop=True)

In [122]:
df_places[df_places.Coordinates.isnull()]

,Bundesland,Ort,Coordinates,Adresses
19,STir.,Reschen am See,None,None
215,OTir.,Lavant,None,None
290,NTir.,Feichten im Kaunertal (Feuchten),None,None
344,NTir.,Freundsheim,None,None
345,NTir.,Untermieming $Untermiening$,None,None
353,NTir.,$Silltall; Anm.: inoffizielle Bezeichnung für ...,None,None
446,NTir.,Thiersee $Tiersee$,None,None
484,NTir.,Aschau bei Kirchberg,None,None
552,Kä.,Nölbing,None,None
571,Kä.,Reisach im Gailtail,None,None


Testing with a missing field...

In [121]:
location = geolocator.geocode('Scheiblingskirchen')
print(location.address)
address = location.address
print((location.latitude, location.longitude))
latitude, longitude = location.latitude, location.longitude
print(location.raw)

AttributeError: 'NoneType' object has no attribute 'address'

In [123]:
df_places[df_places.Coordinates.isnull()].index

Int64Index([  19,  215,  290,  344,  345,  353,  446,  484,  552,  571,  580,
             830,  854, 1037, 1199, 1241, 1281, 1293, 1332, 1615, 1773, 1865,
            1984, 2008, 2012, 2265, 2305, 2566, 2971],
           dtype='int64')

In [125]:
geoinfo_missing = []
for ort in df_places[df_places.Coordinates.isnull()].Ort:
    geoinfo_missing.append(get_coordinates(ort))

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [126]:
print(len(geoinfo_missing))
print(len(df_places[df_places.Coordinates.isnull()]))

29
29


In [106]:
idx = df_places[df_places.Coordinates.isnull()].index
df_places.loc[idx, 'Coordinates'] = [x[0] if x else None for x in geoinfo_missing]

In [107]:
df_places[df_places.Adresses.isnull()]

,Bundesland,Ort,Coordinates,Adresses
19,STir.,Reschen am See,None,None
215,OTir.,Lavant,None,None
290,NTir.,Feichten im Kaunertal (Feuchten),None,None
344,NTir.,Freundsheim,None,None
345,NTir.,Untermieming $Untermiening$,None,None
353,NTir.,$Silltall; Anm.: inoffizielle Bezeichnung für ...,None,None
446,NTir.,Thiersee $Tiersee$,None,None
484,NTir.,Aschau bei Kirchberg,None,None
552,Kä.,Nölbing,None,None
571,Kä.,Reisach im Gailtail,None,None


In [127]:
geoinfo_missing = []
for ort in df_places[df_places.Adresses.isnull()].Ort:
    geoinfo_missing.append(get_coordinates(ort))

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [128]:
print(len(geoinfo_missing))
print(len(df_places[df_places.Adresses.isnull()]))

29
29


In [110]:
idx = df_places[df_places.Adresses.isnull()].index
df_places.loc[idx, 'Adresses'] = [x[1] if x else None for x in geoinfo_missing]

In [129]:
df_places.loc[df_places[df_places.Coordinates.isnull()].index]

,Bundesland,Ort,Coordinates,Adresses
19,STir.,Reschen am See,None,None
215,OTir.,Lavant,None,None
290,NTir.,Feichten im Kaunertal (Feuchten),None,None
344,NTir.,Freundsheim,None,None
345,NTir.,Untermieming $Untermiening$,None,None
353,NTir.,$Silltall; Anm.: inoffizielle Bezeichnung für ...,None,None
446,NTir.,Thiersee $Tiersee$,None,None
484,NTir.,Aschau bei Kirchberg,None,None
552,Kä.,Nölbing,None,None
571,Kä.,Reisach im Gailtail,None,None


In [3]:
df_places.to_pickle("./df_places_plus.pkl")

NameError: name 'df_places' is not defined

In [135]:
#https://docs.python.org/2.4/lib/standard-encodings.html
df_places.to_csv("./df_places.csv", encoding='utf_16')

Showing places in a map

In [4]:
df_places = pd.read_pickle("./df_places_plus.pkl")
df_places.head(10)

,Bundesland,Ort,Coordinates,Adresses
0,W.,Altmannsdorf,"(10.4342455, 49.9358878)","Altmannsdorf, Gerolzhofen (VGem), Landkreis Sc..."
1,W.,Aspern,"(16.4847508, 48.2181854)","Aspern, Donaustadt, Wien, 1220, Österreich"
2,W.,Atzgersdorf,"(16.2954875, 48.1469838)","Atzgersdorf, Liesing, Wien, 1230, Österreich"
3,W.,Breitensee,"(10.5435061, 50.3531716)","Breitensee, Bad Königshofen i. Grabfeld (VGem)..."
4,W.,Favoriten,"(16.3828218364771, 48.1529937)","Favoriten, Wien, 1100, Österreich"
5,W.,Gersthof,"(16.3237393, 48.2341382)","Gersthof, Währing, Wien, 1180, Österreich"
6,W.,Hernals,"(16.2842142186391, 48.2354031)","Hernals, Wien, 1170, Österreich"
7,W.,Hietzing,"(16.2529858155656, 48.17854095)","Hietzing, Wien, 1130, Österreich"
8,W.,Inzersdorf,"(16.3527158778638, 48.14397215)","KG Inzersdorf, Liesing, Wien, 1230, Österreich"
9,W.,Kalksburg,"(16.2457624, 48.1376728)","Kalksburg, Liesing, Wien, 1230, Österreich"


In [5]:
df_places.Coordinates[1][-1::-1]

(48.2181854, 16.4847508)

In [17]:
#https://ipyleaflet.readthedocs.io/en/latest/
#https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html
#https://github.com/jupyter-widgets/ipyleaflet/issues/64

#layer = WMSLayer(url="https://demo.boundlessgeo.com/geoserver/ows?", layers="nasa:bluemarble")
layer = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2018-09-30")
center = (47,14)

mapplaces = Map(#layers=(layer,),
                center=center,
                zoom=6)

mapplaces.layout.width = '100%'
mapplaces.layout.height = '800px'

markers = []
for location in df_places.Coordinates:
    if location:
        markers.append(Marker(location=location[-1::-1], draggable=False,))
                       
marker_cluster = MarkerCluster(markers=tuple(markers))                       
mapplaces.add_layer(marker_cluster);


mapplaces

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

Reading the XML Data files:

In [18]:
xmlfile =  './TEI-XML-2018/bel_d-tei-02/d157_qdb-TEI-02.xml'
#xmlfile = open(xmlfile, mode='r', encoding='utf-8').read()
namespaces = {'TEI': 'http://www.tei-c.org/ns/1.0'} # add more as needed

Parsing the XML tree

In [19]:
xmldoc = minidom.parse(xmlfile)  #xmldoc = ET.XML(xmlfile)
bs = BeautifulSoup(open(xmlfile, encoding="utf8"), 'xml')
tree = ET.parse(xmlfile)

In [20]:
root = tree.getroot()
print(root)

<Element '{http://www.tei-c.org/ns/1.0}TEI' at 0x000001A29C28E368>


In [21]:
my_namespaces = dict([node for _, node in ET.iterparse(StringIO(open(xmlfile, mode='r', encoding='utf-8').read()), events=['start-ns'])])
print(my_namespaces)

{'': 'http://www.tei-c.org/ns/1.0'}


In [22]:
for child in root:
    print(child.tag, child.attrib)

{http://www.tei-c.org/ns/1.0}teiHeader {}
{http://www.tei-c.org/ns/1.0}text {}


In [23]:
set([elem.tag for elem in root.iter()])

{'{http://www.tei-c.org/ns/1.0}TEI',
 '{http://www.tei-c.org/ns/1.0}att',
 '{http://www.tei-c.org/ns/1.0}bibl',
 '{http://www.tei-c.org/ns/1.0}body',
 '{http://www.tei-c.org/ns/1.0}certainty',
 '{http://www.tei-c.org/ns/1.0}cit',
 '{http://www.tei-c.org/ns/1.0}date',
 '{http://www.tei-c.org/ns/1.0}def',
 '{http://www.tei-c.org/ns/1.0}edition',
 '{http://www.tei-c.org/ns/1.0}editionStmt',
 '{http://www.tei-c.org/ns/1.0}editorialDecl',
 '{http://www.tei-c.org/ns/1.0}encodingDesc',
 '{http://www.tei-c.org/ns/1.0}entry',
 '{http://www.tei-c.org/ns/1.0}etym',
 '{http://www.tei-c.org/ns/1.0}fileDesc',
 '{http://www.tei-c.org/ns/1.0}form',
 '{http://www.tei-c.org/ns/1.0}geoDecl',
 '{http://www.tei-c.org/ns/1.0}gi',
 '{http://www.tei-c.org/ns/1.0}gram',
 '{http://www.tei-c.org/ns/1.0}gramGrp',
 '{http://www.tei-c.org/ns/1.0}idno',
 '{http://www.tei-c.org/ns/1.0}interp',
 '{http://www.tei-c.org/ns/1.0}label',
 '{http://www.tei-c.org/ns/1.0}listPlace',
 '{http://www.tei-c.org/ns/1.0}listPrefixDe

In [25]:
places = []
for elem in root.iter('{http://www.tei-c.org/ns/1.0}place'):
    # print(elem.attrib)
    # print(elem.attrib['type'])
    places.append(elem.attrib['type'])
freq_places = collections.Counter(places)
freq_places.most_common()

[('Bundesland', 891),
 ('Großregion', 797),
 ('Kleinregion', 783),
 ('Gemeinde', 696),
 ('Ort', 696)]

Dictionary Approach

In [26]:
xml2dictfile = xmltodict.parse((open(xmlfile, mode='r', encoding='utf-8').read()))

In [27]:
print(xml2dictfile['TEI'].keys())

odict_keys(['@xmlns', 'teiHeader', 'text'])


In [28]:
xml2dictfile['TEI']['@xmlns']
#xml2dictfile['TEI']['teiHeader']

'http://www.tei-c.org/ns/1.0'

In [29]:
print(len(xml2dictfile['TEI']['text']['body']['entry']))
xml2dictfile['TEI']['text']['body']['entry'][0].keys()

1991


odict_keys(['@xml:id', '@xml:lang', 'form', 'gramGrp', 'sense', 'cit', 'note', 'ref', 'etym', 'usg'])

In [30]:
xml2dictfile['TEI']['text']['body']['entry'][0]

OrderedDict([('@xml:id', 'd157_qdb-d1e2'),
             ('@xml:lang', 'bar'),
             ('form',
              [OrderedDict([('@type', 'hauptlemma'), ('orth', 'Tin')]),
               OrderedDict([('@type', 'lautung'),
                            ('@n', '1'),
                            ('pron',
                             [OrderedDict([('@notation', 'tustep'),
                                           ('#text', '|A t~-in')]),
                              OrderedDict([('@notation', 'ipa'),
                                           ('@resp', '#JB'),
                                           ('@change', '01'),
                                           ('#text', '|A t~-in')])]),
                            ('gramGrp', OrderedDict([('gram', '[m,sg+U]')]))]),
               OrderedDict([('@type', 'lautung'),
                            ('@n', '2'),
                            ('pron',
                             [OrderedDict([('@notation', 'tustep'),
                              

In [49]:
datapath = './TEI-XML-2018/'
onlydirs = [f for f in os.listdir(datapath) if os.path.isdir(os.path.join(datapath,f)) and f.startswith('bel')]
onlydirs

['bel_d-tei-02',
 'bel_e-tei-02',
 'bel_f-tei-02',
 'bel_g-tei-02',
 'bel_h-tei-02',
 'bel_i-j-tei-02',
 'bel_k-tei-02',
 'bel_l-tei-02',
 'bel_m-tei-02',
 'bel_n-o-p-tei-02',
 'bel_q-r-tei-02',
 'bel_s-tei-02',
 'bel_u-w-tei-02',
 'bel_z-tei-02']

In [53]:
onlyfiles = []
for folder in onlydirs:
    for filex in [f for f in os.listdir(os.path.join(datapath, folder)) if os.path.isfile(os.path.join(datapath, folder, f)) \
                                       and not f.startswith('.')]:
        onlyfiles.append('./{}/{}'.format(folder, filex))


print('Files in the folder:')
for i, w in enumerate(onlyfiles[0:]):
    print(i+1, '--' ,w)

Files in the folder:
1 -- ./bel_d-tei-02/d154_156_qdb-TEI-02.xml
2 -- ./bel_d-tei-02/d157_qdb-TEI-02.xml
3 -- ./bel_d-tei-02/d158_qdb-TEI-02.xml
4 -- ./bel_d-tei-02/d159_qdb-TEI-02.xml
5 -- ./bel_d-tei-02/d160_qdb-TEI-02.xml
6 -- ./bel_d-tei-02/d161_qdb-TEI-02.xml
7 -- ./bel_d-tei-02/d162_qdb-TEI-02.xml
8 -- ./bel_d-tei-02/d163_qdb-TEI-02.xml
9 -- ./bel_d-tei-02/d164_qdb-TEI-02.xml
10 -- ./bel_d-tei-02/d165_qdb-TEI-02.xml
11 -- ./bel_d-tei-02/d166_qdb-TEI-02.xml
12 -- ./bel_d-tei-02/d167_qdb-TEI-02.xml
13 -- ./bel_d-tei-02/d168_qdb-TEI-02.xml
14 -- ./bel_d-tei-02/d169_qdb-TEI-02.xml
15 -- ./bel_d-tei-02/d170_qdb-TEI-02.xml
16 -- ./bel_d-tei-02/d171_qdb-TEI-02.xml
17 -- ./bel_d-tei-02/d172_qdb-TEI-02.xml
18 -- ./bel_d-tei-02/d173_qdb-TEI-02.xml
19 -- ./bel_d-tei-02/d174_qdb-TEI-02.xml
20 -- ./bel_d-tei-02/d175_qdb-TEI-02.xml
21 -- ./bel_d-tei-02/d176_qdb-TEI-02.xml
22 -- ./bel_d-tei-02/d177_qdb-TEI-02.xml
23 -- ./bel_d-tei-02/d178_qdb-TEI-02.xml
24 -- ./bel_d-tei-02/d179_qdb-TEI-02.xml


590 -- ./bel_s-tei-02/s744_qdb-TEI-02.xml
591 -- ./bel_s-tei-02/s745_qdb-TEI-02.xml
592 -- ./bel_s-tei-02/s746_qdb-TEI-02.xml
593 -- ./bel_s-tei-02/s747_qdb-TEI-02.xml
594 -- ./bel_s-tei-02/s748_qdb-TEI-02.xml
595 -- ./bel_s-tei-02/s749_qdb-TEI-02.xml
596 -- ./bel_s-tei-02/s750_qdb-TEI-02.xml
597 -- ./bel_s-tei-02/s751_qdb-TEI-02.xml
598 -- ./bel_s-tei-02/s752_qdb-TEI-02.xml
599 -- ./bel_s-tei-02/s753_qdb-TEI-02.xml
600 -- ./bel_s-tei-02/s754_qdb-TEI-02.xml
601 -- ./bel_s-tei-02/s755_qdb-TEI-02.xml
602 -- ./bel_s-tei-02/s756_qdb-TEI-02.xml
603 -- ./bel_s-tei-02/s757_qdb-TEI-02.xml
604 -- ./bel_s-tei-02/s758_qdb-TEI-02.xml
605 -- ./bel_s-tei-02/s759_qdb-TEI-02.xml
606 -- ./bel_s-tei-02/s760_qdb-TEI-02.xml
607 -- ./bel_s-tei-02/s761_qdb-TEI-02.xml
608 -- ./bel_s-tei-02/s762_qdb-TEI-02.xml
609 -- ./bel_s-tei-02/s763_qdb-TEI-02.xml
610 -- ./bel_s-tei-02/s764_qdb-TEI-02.xml
611 -- ./bel_s-tei-02/s765_qdb-TEI-02.xml
612 -- ./bel_s-tei-02/s766_qdb-TEI-02.xml
613 -- ./bel_s-tei-02/s767_qdb-TEI

In [35]:
places_lemmas = collections.defaultdict(list)

for idx, entry in enumerate(xml2dictfile['TEI']['text']['body']['entry']):
    #print(idx)
    places = []
    lemmas = []
    if 'usg' in entry.keys():
        if isinstance(entry['usg'], list):
            for local in entry['usg']:
                try:
                    #print(local['placeName']['#text'])
                    places.append(local['placeName']['#text'])
                except:
                    #print(local['placeName'])
                    places.append(local['placeName'])
        else:
            places = []
            try:
                #print(entry['usg']['placeName']['#text']);
                places.append(entry['usg']['placeName']['#text'])
            except:
                #print(entry['usg']['placeName']);
                places.append(entry['usg']['placeName'])
                
    if 'form' in entry.keys():            
        if isinstance(entry['form'], list):
            for elem in entry['form']:
                if elem['@type'] == 'hauptlemma' and isinstance(elem['orth'], str):          
                    #print(elem['orth'])
                    lemmas.append(elem['orth'])
                elif elem['@type'] == 'hauptlemma' and isinstance(elem['orth'], list):
                    for subelem in elem['orth']:
                        if subelem['@type'] == 'normalized':
                            #print(subelem['#text'])
                            lemmas.append(subelem['#text'])
        else:
            if isinstance(entry['form']['orth'], list):
                for elem in entry['form']['orth']:
                    if elem['@type'] == 'normalized':
                        #print(elem['#text'])
                        lemmas.append(elem['#text'])
            else:
                #print(entry['form']['orth'])
                lemmas.append(entry['form']['orth'])
    if places and lemmas:
        for place in places:
            places_lemmas[place].extend(lemmas)
                
    #if 'cit' in entry.keys():
    #    print(entry['cit']) #['ref'])
    #    print(entry['cit'].keys())
  
    #print(lemmas, places)

In [39]:
freqdist = {}
for key, value in places_lemmas.items():
    freqdist[key] = collections.Counter(value)

In [41]:
freqdist['(Umg.) Herzogenb-g. NÖ'].most_common(10)

[('Ding', 10),
 ('dirékt', 3),
 ('Tinte', 2),
 ('tippen', 2),
 ('Diréktor', 2),
 ('Dinamít', 1),
 ('Dinamo', 1),
 ('Dinastí', 1),
 ('dinástisch', 1),
 ('dingen', 1)]

In [43]:
#https://ipyleaflet.readthedocs.io/en/latest/
#https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html
#https://github.com/jupyter-widgets/ipyleaflet/issues/64

#layer = WMSLayer(url="https://demo.boundlessgeo.com/geoserver/ows?", layers="nasa:bluemarble")
layer = basemap_to_tiles(basemaps.NASAGIBS.ModisTerraTrueColorCR, "2018-09-30")
center = (47,14)

mapplaces = Map(#layers=(layer,),
                center=center,
                zoom=6)

mapplaces.layout.width = '100%'
mapplaces.layout.height = '800px'

markers = []
for idx in df_places.index:
    if df_places.loc[idx, 'Coordinates']:
        markers.append(Marker(location=location[-1::-1], 
                              draggable=False, 
                              title=freqdist[df_places.loc[idx,'Ort']].most.common(5)))
                       
marker_cluster = MarkerCluster(markers=tuple(markers))                       
mapplaces.add_layer(marker_cluster);


mapplaces

KeyError: 'Altmannsdorf'

In [44]:
freqdist.keys()

dict_keys(['Matrei/O. OTir.', 'Salurn', '(Umg.) Herzogenb-g. NÖ', 'Königswerth Egerld.', 'Marling', 'Malta Kä.', 'Wien', 'Böwald', 'STir.', 'alt in Laurein', '(Umg./Bz.) Braunau OÖ', 'in der Pladen benachbarten Sprachinsel Zahre', 'Pladen', 'W.', 'Weibern OÖ', 'Ulrichsbg. OÖ', 'Weyer-Gaflenz Oö.', 'Poysdf. NÖ', 'Radschin in Gm. Kautzen NÖ [2o/1.]', 'Spital a. Pyhrn', 'Salzburg', 'Plattetschl. BöW', 'Reidling in Sitzenbg.-Reidling NÖ', 'Weident. WRum.', 'Saalfelden', 'U', 'Fusch Sa.', 'Fendels NTir.', 'Thiers. NTir.', 'St.Johann/T. St.: obPölst.:Pölst.:nördl.mObMurgeb.:sbair.ObSt.  Aufn.Krz.:1924', 'St.Anton NÖ', 'Kiens STir.', 'Pottendf. NÖ', 'cimbr.', 'velt.', 'Schlagen in Gm. Gmunden OÖ', 'Hohenfurth SBö.', 'Egerland', '(Leimsgrub u. Moarhäuser in Gm.) Schönbg. BöW', 'Plan Egerld.', 'Kufstn. NTir.', 'Niederndf./NTir. undThiers. NTir.', 'Neustadtl NÖ  FbB.TILL· (19xx)?', '(Ebene Reichenau in Gm.) Reichenau Kä.', 'Mühlhm. OÖ', 'Niederkappel OÖ  Fb.Leitenbauer (Witzersdf.)', 'St.Johann 